# Segmenting and Clustering Neighborhoods in Toronto

### Lybraries

In [1]:
import pandas as pd # library to process data as dataframes
import requests
import geocoder # import geocoder
from bs4 import BeautifulSoup

import bs4 as bs
import urllib.request

print('Libraries imported.')

Libraries imported.


### Building the dataframe for the Toronto Neighbourhoods using BeautifulSoup.


In [2]:
# Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
source = urllib.request.urlopen(url).read()
soup = bs.BeautifulSoup(source,'lxml')


table = soup.find('table')
table_rows = table.find_all('tr')


line = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        line.append(row)
        
df = pd.DataFrame(line, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,None
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [3]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
df.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
# More than one neighborhood can exist in one postal code area.
# Wikipedia manage this case, using as a separator for Neighbourhood "/"
# Replace "/" with ","

df['Neighbourhood'] = df['Neighbourhood'].str.replace('/',',')
df.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern , Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill , Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
# There is not any Neighboorhodd with 'Not assignesd' value'
count = df[df.Neighbourhood == 'Not assigned'].count()
count

PostalCode       0
Borough          0
Neighbourhood    0
dtype: int64

In [6]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df.shape

(103, 3)

### Project Part 2: Getting the latitude and the longitude coordinates of each neighborhood.


In [7]:
# geocoder, at the present, doesn't work, so I'm going to use csv file

#df['Latitude'] = ''
#df['Longitude'] = ''

# loop through the postal code get latitude and longitude
#for index, data in df.iterrows():
#    lat_lng_coords = None
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(data['PostalCode']))
#        lat_lng_coords = g.latlng
#    data['Latitude'] = lat_lng_coords[0]
#    data['Longitude'] = lat_lng_coords[1]
#    print ('PostalCode:', data['PostalCode'], 'Latitude:', data['Latitude'], 'Longitude:', data['Longitude'])


In [8]:
csv_ll = pd.read_csv('http://cocl.us/Geospatial_data')

In [9]:
csv_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
csv_ll.rename(columns={'Postal Code': 'PostalCode'}, inplace = True)
df = pd.merge(df, csv_ll, on ='PostalCode')

In [12]:
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
